In [142]:
import numpy as np
from StringIO import StringIO
import pandas as pd
from __future__ import division
import matplotlib.pyplot as plt
import urllib2,base64
import cPickle as pickle
from collections import Counter
from collections import defaultdict
from collections import OrderedDict
from __future__ import division
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import geoplotlib as gp
import scipy as sp
import pysal as ps
import datetime
import operator
import re
import os
import os.path
import requests, time
import urllib, json
import csv
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
tls.set_credentials_file(username='ShawnHou', api_key='OohmqCJaPfb6p6xo4pDF')
from geoplotlib.utils import BoundingBox
from sklearn import neighbors, datasets,linear_model,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import scale
from pylab import *
from scipy import stats
from sklearn.neighbors import DistanceMetric
from urllib2 import urlopen, Request
from pprint import pprint
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
from collections import defaultdict
from operator import itemgetter
from pushover import init, Client
from scipy.spatial import KDTree
from calendar import monthrange
import tarfile
%matplotlib inline

## Motivation 

https://www.technologyreview.com/s/608961/billion-tweet-study-proves-we-write-happier-messages-when-the-weather-is-good/

## Data processing

In [143]:
temp_stations = pd.read_csv(os.getcwd()+'\\data\\data option one\\weather_station_list.csv',skipinitialspace=True)
temp_stations['GHCND']=temp_stations.GHCND.str.strip()
temp_stations=temp_stations[temp_stations.GHCND.str.contains("US")] ##Discard non US stations
temp_stations_clean=temp_stations[np.abs(temp_stations.LON-temp_stations.LON.mean())<=(5*temp_stations.LON.std())] ##discard OUTLIER by location

print "include outliner, size:",temp_stations.shape
print "exclude outliner, size:",temp_stations_clean.shape

include outliner, size: (56918, 6)
exclude outliner, size: (56911, 6)


In [144]:
date_rng_dict=dict()
for year in range(2017,1985,-1):
    if year%4==0:
        date_rng=[d.strftime('%Y%m%d') for d in pd.date_range(str(year)+'-1-1', periods=366, freq='D')] #range from 1-1 to 12-31
    else:
        date_rng=[d.strftime('%Y%m%d') for d in pd.date_range(str(year)+'-1-1', periods=365, freq='D')] #range from 1-1 to 12-31
    date_rng_dict[year]=date_rng

date_rng_dict=OrderedDict(sorted(date_rng_dict.items(), key=lambda t: t[0]))

In [145]:
metrics=year_df['ATTRIBUTE'].unique().tolist()
print "All different weather metrics: ",metrics 

All different weather metrics:  ['TMAX', 'TMIN', 'PRCP', 'SNOW', 'SNWD', 'TAVG', 'ACMH', 'ACSH', 'AWND', 'FMTM', 'PGTM', 'TSUN', 'WDFG', 'WDFM', 'WSFG', 'WSFM', 'WT01', 'WT06', 'WT16', 'WT17', 'WDF1', 'WSF1', 'WT03', 'WESD', 'TOBS', 'EVAP', 'MNPN', 'MXPN', 'WDMV', 'SN02', 'SX02', 'WT04', 'WT08', 'WT18', 'SN01', 'SN03', 'SX01', 'SX03', 'WT09', 'SN12', 'SN32', 'SX12', 'SX32', 'WT02', 'WT11', 'WT14', 'SN31', 'SN33', 'SN51', 'SN61', 'SX31', 'SX33', 'SX51', 'SX61', 'WT22', 'DATX', 'MDTX', 'WT12', 'SN52', 'SN72', 'SN81', 'SN82', 'SN83', 'SX52', 'SX72', 'SX81', 'SX82', 'SX83', 'WT05', 'SN53', 'SX53', 'WT15', 'DAPR', 'DWPR', 'MDPR', 'WT07', 'DATN', 'MDTN', 'MDSF', 'SN11', 'SN13', 'SX11', 'SX13', 'DASF', 'WT10', 'SN21', 'SN22', 'SN23', 'SX21', 'SX22', 'SX23', 'DAWM', 'MDWM', 'DAEV', 'MDEV']


The five core elements are:

           PRCP = Precipitation (tenths of mm)
   	   SNOW = Snowfall (mm)
	   SNWD = Snow depth (mm)
           TMAX = Maximum temperature (tenths of degrees C)
           TMIN = Minimum temperature (tenths of degrees C)
	   
	   The other elements are:
	   
	   ACMC = Average cloudiness midnight to midnight from 30-second 
	          ceilometer data (percent)
	   ACMH = Average cloudiness midnight to midnight from 
	          manual observations (percent)
           ACSC = Average cloudiness sunrise to sunset from 30-second 
	          ceilometer data (percent)
	   ACSH = Average cloudiness sunrise to sunset from manual 
	          observations (percent)
           AWDR = Average daily wind direction (degrees)
	   AWND = Average daily wind speed (tenths of meters per second)
	   DAEV = Number of days included in the multiday evaporation
	          total (MDEV)
	   DAPR = Number of days included in the multiday precipiation 
	          total (MDPR)
           DASF = Number of days included in the multiday snowfall 
	          total (MDSF)		  
	   DATN = Number of days included in the multiday minimum temperature 
	         (MDTN)
	   DATX = Number of days included in the multiday maximum temperature 
	          (MDTX)
           DAWM = Number of days included in the multiday wind movement
	          (MDWM)
	   DWPR = Number of days with non-zero precipitation included in 
	          multiday precipitation total (MDPR)
	   EVAP = Evaporation of water from evaporation pan (tenths of mm)
	   FMTM = Time of fastest mile or fastest 1-minute wind 
	          (hours and minutes, i.e., HHMM)
	   FRGB = Base of frozen ground layer (cm)
	   FRGT = Top of frozen ground layer (cm)
	   FRTH = Thickness of frozen ground layer (cm)
	   GAHT = Difference between river and gauge height (cm)
	   MDEV = Multiday evaporation total (tenths of mm; use with DAEV)
	   MDPR = Multiday precipitation total (tenths of mm; use with DAPR and 
	          DWPR, if available)
	   MDSF = Multiday snowfall total 
	   MDTN = Multiday minimum temperature (tenths of degrees C; use with 
	          DATN)
	   MDTX = Multiday maximum temperature (tenths of degress C; use with 
	          DATX)
	   MDWM = Multiday wind movement (km)
           MNPN = Daily minimum temperature of water in an evaporation pan 
	         (tenths of degrees C)
           MXPN = Daily maximum temperature of water in an evaporation pan 
	         (tenths of degrees C)
	   PGTM = Peak gust time (hours and minutes, i.e., HHMM)
	   PSUN = Daily percent of possible sunshine (percent)
	   SN*# = Minimum soil temperature (tenths of degrees C)
	          where * corresponds to a code
	          for ground cover and # corresponds to a code for soil 
		  depth.  
		  
		  Ground cover codes include the following:
		  0 = unknown
		  1 = grass
		  2 = fallow
		  3 = bare ground
		  4 = brome grass
		  5 = sod
		  6 = straw multch
		  7 = grass muck
		  8 = bare muck
		  
		  Depth codes include the following:
		  1 = 5 cm
		  2 = 10 cm
		  3 = 20 cm
		  4 = 50 cm
		  5 = 100 cm
		  6 = 150 cm
		  7 = 180 cm
		  
	   SX*# = Maximum soil temperature (tenths of degrees C) 
	          where * corresponds to a code for ground cover 
		  and # corresponds to a code for soil depth. 
		  See SN*# for ground cover and depth codes. 
           TAVG = Average temperature (tenths of degrees C)
	          [Note that TAVG from source 'S' corresponds
		   to an average for the period ending at
		   2400 UTC rather than local midnight]
           THIC = Thickness of ice on water (tenths of mm)	
 	   TOBS = Temperature at the time of observation (tenths of degrees C)
	   TSUN = Daily total sunshine (minutes)
	   WDF1 = Direction of fastest 1-minute wind (degrees)
	   WDF2 = Direction of fastest 2-minute wind (degrees)
	   WDF5 = Direction of fastest 5-second wind (degrees)
	   WDFG = Direction of peak wind gust (degrees)
	   WDFI = Direction of highest instantaneous wind (degrees)
	   WDFM = Fastest mile wind direction (degrees)
           WDMV = 24-hour wind movement (km)	   
           WESD = Water equivalent of snow on the ground (tenths of mm)
	   WESF = Water equivalent of snowfall (tenths of mm)
	   WSF1 = Fastest 1-minute wind speed (tenths of meters per second)
	   WSF2 = Fastest 2-minute wind speed (tenths of meters per second)
	   WSF5 = Fastest 5-second wind speed (tenths of meters per second)
	   WSFG = Peak gust wind speed (tenths of meters per second)
	   WSFI = Highest instantaneous wind speed (tenths of meters per second)
	   WSFM = Fastest mile wind speed (tenths of meters per second)
	   WT** = Weather Type where ** has one of the following values:
	   
                  01 = Fog, ice fog, or freezing fog (may include heavy fog)
                  02 = Heavy fog or heaving freezing fog (not always 
		       distinquished from fog)
                  03 = Thunder
                  04 = Ice pellets, sleet, snow pellets, or small hail 
                  05 = Hail (may include small hail)
                  06 = Glaze or rime 
                  07 = Dust, volcanic ash, blowing dust, blowing sand, or 
		       blowing obstruction
                  08 = Smoke or haze 
                  09 = Blowing or drifting snow
                  10 = Tornado, waterspout, or funnel cloud 
                  11 = High or damaging winds
                  12 = Blowing spray
                  13 = Mist
                  14 = Drizzle
                  15 = Freezing drizzle 
                  16 = Rain (may include freezing rain, drizzle, and
		       freezing drizzle) 
                  17 = Freezing rain 
                  18 = Snow, snow pellets, snow grains, or ice crystals
                  19 = Unknown source of precipitation 
                  21 = Ground fog 
                  22 = Ice fog or freezing fog
		  
            WV** = Weather in the Vicinity where ** has one of the following 
	           values:
		   
		   01 = Fog, ice fog, or freezing fog (may include heavy fog)
		   03 = Thunder
		   07 = Ash, dust, sand, or other blowing obstruction
		   18 = Snow or ice crystals
		   20 = Rain or snow shower

In [150]:
temp_stations_clean.head()

,GHCND,LAT,LON,ELEV,ST,NAME
0,US009052008,43.7333,-96.6333,482.0,SD,SIOUX FALLS (ENVIRON. CANADA)
1,US10adam001,40.5680,-98.5069,598.0,NE,JUNIATA 1.5 S
2,US10adam002,40.5093,-98.5493,601.1,NE,JUNIATA 6.0 SSW
3,US10adam003,40.4663,-98.6537,615.1,NE,HOLSTEIN 0.1 NW
4,US10adam004,40.4798,-98.4026,570.0,NE,AYR 3.5 NE


In [ ]:
wanted_metrics=[]

In [161]:
years_stat=dict()
for YEAR in range(2017,1979,-1):
    year_file=os.getcwd()+'\\data\\data option one\\'+str(YEAR)+'.csv.gz'
    year_df = pd.read_csv(year_file, compression='gzip', header=None,usecols = used_columns,
                              dtype=dict(zip(used_columns, dtypes)),sep=',').rename(columns=dict(zip(used_columns, data_columns)))
    #year_station=year_df[ year_df['GHCND'].isin(temp_stations_clean['GHCND'].tolist())
                                 #   & ( year_df['ATTRIBUTE'].isin([metric])) & ( year_df['VALUE']!=-999) ]
    year_station_df=pd.merge(year_df,temp_stations_clean, how='right', on="GHCND")
    grouped_year_station= year_station_df.groupby(['DATE','ATTRIBUTE'])
    for name, group in grouped_year_station:
        group_sup_df=group[['GHCND','VALUE','LAT','LON','ELEV','ST']]
        group_sup_df.to_csv('data\\daily dataframes\\{}.csv'.format(name),header=False, index_label=False)
    years_stat[YEAR]=year_station_df
    print 'Year: ',YEAR, ' - ', year_station_df.info()

Year:  2017  - <class 'pandas.core.frame.DataFrame'>
Int64Index: 24510550 entries, 0 to 24510549
Data columns (total 10 columns):
GHCND        object
DATE         object
ATTRIBUTE    object
VALUE        float64
TIME         object
LAT          float64
LON          float64
ELEV         float64
ST           object
NAME         object
dtypes: float64(4), object(6)
memory usage: 2.0+ GB
 None


In [148]:
years_stat=OrderedDict(sorted(years_stat.items(),key=operator.itemgetter(0),reverse=True))

In [149]:
years_dict_stat=dict()
for year, count in years_stat.items():
    dict_result=dict(count.items())
    years_dict_stat[year]=dict_result
years_dict_stat=OrderedDict(sorted(years_dict_stat.items(),key=operator.itemgetter(0),reverse=True))
with open(metric+"_stat.json",'w') as f:
    json.dump(years_dict_stat,f)

#### Avoid reexecuting following section !!!! Due to connection to remote web service

In [ ]:
year="2016"
url_base = "http://data.fcc.gov/api/block/find?"
county_fips=list()
county_names=list()
block_fips=list()
print temp_stations_clean.shape
i=0
for index, row in temp_stations_clean.iterrows():
    lat=str(row['LAT'])
    lon=str(row['LON'])
    url_geo="latitude="+lat+"&"+"longitude="+lon+"&format=json"
    url=url_base+url_geo
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    county_fips.append(data['County']['FIPS'])
    county_names.append(data['County']['name'])
    block_fips.append(data['Block']['FIPS'])
    i=i+1
    if i%1000==0:
        print i
    if i%15000==0:
        time.sleep(600)
        

county_fips_s = pd.Series(county_fips)
county_name_s = pd.Series(county_names)
block_fips_s = pd.Series(block_fips)
temp_stations_clean['COUNTY_CODE'] = county_fips_s.values
temp_stations_clean['COUNTY_NAME'] = county_name_s.values
temp_stations_clean['BLOCK_CODE'] = block_fips_s.values

In [39]:
temp_stations_clean.to_csv('data\\daily dataframe\\cleaned_weather_station_list_withCounty.csv', index=False,encoding="UTF-8")

#### Get average tem of each county in a specific date range (Some counties are missing)

In [5]:
temp_stations_clean=pd.read_csv('data\\daily dataframe\\cleaned_weather_station_list_withCounty.csv',skipinitialspace=True)

In [17]:
temp_stations_clean.dtypes

GHCND           object
LAT            float64
LON            float64
ELEV           float64
ST              object
NAME            object
COUNTY_CODE    float64
COUNTY_NAME     object
BLOCK_CODE     float64
dtype: object

In [21]:
daily_county_avg=defaultdict(dict)
for date in date_rng:
    reported_data_stations=yearly_data[(yearly_data['DATE']==date) & (yearly_data['ATTRIBUTE'].isin(["TMAX"])) & (yearly_data['VALUE']!=-999)]
    reported_full_stations=pd.merge(temp_stations_clean, reported_data_stations, on='GHCND', how='inner')
    reported_full_stations=reported_full_stations.groupby(['COUNTY_CODE'])['VALUE'].mean().reset_index().sort_values('COUNTY_CODE')
    reported_full_stations.VALUE=reported_full_stations.VALUE.round(2)
    reported_full_stations['COUNTY_CODE']=reported_full_stations['COUNTY_CODE'].astype(int).astype(str)
    for index, row in reported_full_stations.iterrows():
        daily_county_avg[row['COUNTY_CODE'].zfill(5)][date]=row['VALUE']
    
    print date, " finished!"

        
daily_county_avg=OrderedDict(sorted(daily_county_avg.items()))
    

#print daily_county_avg

20160101  finished!
20160102  finished!
20160103  finished!
20160104  finished!
20160105  finished!
20160106  finished!
20160107  finished!
20160108  finished!
20160109  finished!
20160110  finished!
20160111  finished!
20160112  finished!
20160113  finished!
20160114  finished!
20160115  finished!
20160116  finished!
20160117  finished!
20160118  finished!
20160119  finished!
20160120  finished!
20160121  finished!
20160122  finished!
20160123  finished!
20160124  finished!
20160125  finished!
20160126  finished!
20160127  finished!
20160128  finished!
20160129  finished!
20160130  finished!
20160131  finished!
20160201  finished!
20160202  finished!
20160203  finished!
20160204  finished!
20160205  finished!
20160206  finished!
20160207  finished!
20160208  finished!
20160209  finished!
20160210  finished!
20160211  finished!
20160212  finished!
20160213  finished!
20160214  finished!
20160215  finished!
20160216  finished!
20160217  finished!
20160218  finished!
20160219  finished!


#### KNN Handles

In [27]:
daily_county_avg_knn_dfs=dict()
for date in date_rng:
    reported_data_knn_stations=yearly_data[(yearly_data['DATE']==date) & (yearly_data['ATTRIBUTE'].isin(["TMAX"])) & (yearly_data['VALUE']!=-999)]
    reported_full_knn_stations=pd.merge(temp_stations_clean, reported_data_knn_stations, on='GHCND', how='inner')
    reported_avg_knn_counties=reported_full_knn_stations.groupby(['COUNTY_CODE'])['VALUE'].mean().reset_index().sort_values('COUNTY_CODE')
    reported_avg_knn_counties.VALUE=reported_avg_knn_counties.VALUE.round(2)
   
    reported_avg_knn_counties['COUNTY_CODE'] = reported_avg_knn_counties['COUNTY_CODE'].apply(lambda x: format(int(x), '05d'))  
    
    #KNN models on missing data
    reported_train_knn_X=reported_full_knn_stations[['LAT','LON']].as_matrix()
    reported_train_knn_Y=reported_full_knn_stations[['VALUE']].as_matrix()
    neigh = KNeighborsClassifier(n_neighbors=5, weights='distance')
    neigh.fit(reported_train_knn_X, reported_train_knn_Y.ravel()) 
    
    #for county in missing_county_list:
    for county in missing_county_list: 
        if county not in reported_avg_knn_counties['COUNTY_CODE'].tolist():
            county_lon = -float(usa_counties_df[usa_counties_df['FIPS']==county]['Longitude'].values[0].decode("utf-8", "ignore")[1:-1])
            county_lat = float(usa_counties_df[usa_counties_df['FIPS']==county]['Latitude'].values[0].decode("utf-8", "ignore")[1:-1])
            pre_tem=neigh.predict([[county_lat, county_lon]])[0]
            append_df = pd.DataFrame([[county, pre_tem.round(2)]],columns=['COUNTY_CODE','VALUE'])
            reported_avg_knn_counties=reported_avg_knn_counties.append(append_df,ignore_index=True) 
            
    #for county in incomplete_county_list:
    for county in incomplete_county_list:
        if county not in reported_avg_knn_counties['COUNTY_CODE'].tolist():
            county_lon = -float(usa_counties_df[usa_counties_df['FIPS']==county]['Longitude'].values[0].decode("utf-8", "ignore")[1:-1])
            county_lat = float(usa_counties_df[usa_counties_df['FIPS']==county]['Latitude'].values[0].decode("utf-8", "ignore")[1:-1])
            pre_tem=neigh.predict([[county_lat, county_lon]])[0]
            append_df = pd.DataFrame([[county, pre_tem.round(2)]],columns=['COUNTY_CODE','VALUE'])
            reported_avg_knn_counties=reported_avg_knn_counties.append(append_df,ignore_index=True) 
            
    print reported_avg_knn_counties.shape, " date: ", date
    
    reported_avg_knn_counties=reported_avg_knn_counties.sort_values('COUNTY_CODE')
    
    daily_county_avg_knn_dfs[date]=reported_avg_knn_counties['VALUE'].tolist()

(3143, 2)  date:  20160101
(3143, 2)  date:  20160102
(3143, 2)  date:  20160103
(3143, 2)  date:  20160104
(3143, 2)  date:  20160105
(3143, 2)  date:  20160106
(3143, 2)  date:  20160107
(3143, 2)  date:  20160108
(3143, 2)  date:  20160109
(3143, 2)  date:  20160110
(3143, 2)  date:  20160111
(3143, 2)  date:  20160112
(3143, 2)  date:  20160113
(3143, 2)  date:  20160114
(3143, 2)  date:  20160115
(3143, 2)  date:  20160116
(3143, 2)  date:  20160117
(3143, 2)  date:  20160118
(3143, 2)  date:  20160119
(3143, 2)  date:  20160120
(3143, 2)  date:  20160121
(3143, 2)  date:  20160122
(3143, 2)  date:  20160123
(3143, 2)  date:  20160124
(3143, 2)  date:  20160125
(3143, 2)  date:  20160126
(3143, 2)  date:  20160127
(3143, 2)  date:  20160128
(3143, 2)  date:  20160129
(3143, 2)  date:  20160130
(3143, 2)  date:  20160131
(3143, 2)  date:  20160201
(3143, 2)  date:  20160202
(3143, 2)  date:  20160203
(3143, 2)  date:  20160204
(3143, 2)  date:  20160205
(3143, 2)  date:  20160206
(

(3143, 2)  date:  20161031
(3143, 2)  date:  20161101
(3143, 2)  date:  20161102
(3143, 2)  date:  20161103
(3143, 2)  date:  20161104
(3143, 2)  date:  20161105
(3143, 2)  date:  20161106
(3143, 2)  date:  20161107
(3143, 2)  date:  20161108
(3143, 2)  date:  20161109
(3143, 2)  date:  20161110
(3143, 2)  date:  20161111
(3143, 2)  date:  20161112
(3143, 2)  date:  20161113
(3143, 2)  date:  20161114
(3143, 2)  date:  20161115
(3143, 2)  date:  20161116
(3143, 2)  date:  20161117
(3143, 2)  date:  20161118
(3143, 2)  date:  20161119
(3143, 2)  date:  20161120
(3143, 2)  date:  20161121
(3143, 2)  date:  20161122
(3143, 2)  date:  20161123
(3143, 2)  date:  20161124
(3143, 2)  date:  20161125
(3143, 2)  date:  20161126
(3143, 2)  date:  20161127
(3143, 2)  date:  20161128
(3143, 2)  date:  20161129
(3143, 2)  date:  20161130
(3143, 2)  date:  20161201
(3143, 2)  date:  20161202
(3143, 2)  date:  20161203
(3143, 2)  date:  20161204
(3143, 2)  date:  20161205
(3143, 2)  date:  20161206
(

In [28]:
#ADD FIPS code to each line of observations as index column. and save data
daily_county_avg_knn_dfs['FIPS']=county_list
daily_county_avg_knn_dfs=pd.DataFrame.from_dict(daily_county_avg_knn_dfs,orient='index').transpose()
daily_county_avg_knn_dfs.sort_index(axis=1, inplace=True)

daily_county_avg_knn_dfs.to_csv('data\\daily dataframe\\county_values_knn.csv', index=False)

In [520]:
daily_county_avg_knn_dfs.dtypes

20160101    object
20160102    object
20160103    object
20160104    object
20160105    object
20160106    object
20160107    object
20160108    object
20160109    object
20160110    object
20160111    object
20160112    object
20160113    object
20160114    object
20160115    object
20160116    object
20160117    object
20160118    object
20160119    object
20160120    object
20160121    object
20160122    object
20160123    object
20160124    object
20160125    object
20160126    object
20160127    object
20160128    object
20160129    object
20160130    object
             ...  
20161203    object
20161204    object
20161205    object
20161206    object
20161207    object
20161208    object
20161209    object
20161210    object
20161211    object
20161212    object
20161213    object
20161214    object
20161215    object
20161216    object
20161217    object
20161218    object
20161219    object
20161220    object
20161221    object
20161222    object
20161223    object
20161224    

### Twitter Data Ingestion

#### 1. LOAD WEATHER KEYWORDS

In [30]:
js = open('data\\daily dataframe\\keyword_map.json').read()
weather_keywords = json.loads(js)
print weather_keywords.keys()

[u'scalding', u'flaming', u'humid', u'hypothermic', u'shivering', u'scorched', u'nippy', u'balmy', u'steamy', u'frigorific', u'antarctic', u'wintry', u'ice-cold', u'shivery', u'sweltry', u'broiling', u'scorching', u'summery', u'icy-cold', u'siberian', u'arid', u'sweaty', u'tropical', u'hot', u'sweat-inducing', u'sweltering', u'bleak', u'ovenlike', u'stifling', u'glacial', u'heat-wave', u'steaming-hot', u'boiling-hot', u'baking-hot', u'afire', u'blistering', u'algid', u'chilled to the bone', u'snowy', u'white-hot', u'heatwave', u'arctic', u'blazing-hot', u'rimy', u'chilled-through', u'stinging-cold', u'freezing', u'warm', u'gelid', u'smoking', u'blizzardy', u'icy', u'conflagrant', u'sizzling', u'frozen-over', u'muggy', u'sub-zero', u'chilled', u'bitterly-cold', u'smoking-hot', u'frozen', u'torrid', u'blazing', u'red-hot', u'chilly', u'bitter-cold', u'parky', u'parching', u'numbing', u'numbing-cold', u'calescent', u'shiver-inducing', u'sultry', u'frosty']


#### 2. LOAD WEATHER COUNTY

In [36]:
temp_stations_tweets=pd.read_csv('data\\daily dataframe\\weather_keyword_counts.tsv',sep='\t', skipinitialspace=True, names=['DATE','RECORDS'], skiprows=1)
temp_stations_tweets.head()

,DATE,RECORDS
0,2016_01_01,"{""hot"": {""16079"": 0.3020231899698423, ""33017"":..."
1,2016_01_02,"{""hot"": {""16079"": 0.20134879331322816, ""33017""..."
2,2016_01_03,"{""hot"": {""16079"": 0.6507980323282503, ""40135"":..."
3,2016_01_04,"{""cold"": {""16079"": 0.03355813221887136, ""40135..."
4,2016_01_05,"{""hot"": {""16079"": 0.44944923901502215, ""40135""..."


In [220]:
daily_county_hottweets_dict=defaultdict(dict)
daily_county_coldtweets_dict=defaultdict(dict)
for index, row in temp_stations_tweets.iterrows():
    record_object=json.loads(row.RECORDS)
    record_date=row.DATE
    record_object['hot']= {format(int(k), '05d'):round(v,2) for k,v in record_object['hot'].items()}
    record_object['cold']= {format(int(k), '05d'):round(v,2) for k,v in record_object['cold'].items()}
    daily_county_hottweets=OrderedDict(sorted(record_object['hot'].items()))
    daily_county_coldtweets=OrderedDict(sorted(record_object['cold'].items()))
    
    
    record_date=record_date.replace('_','');
    
    for county in county_list:
        if county in daily_county_hottweets.keys():
            daily_county_hottweets_dict[record_date][county]=daily_county_hottweets[county]
        if county in daily_county_coldtweets.keys():
            daily_county_coldtweets_dict[record_date][county]=daily_county_coldtweets[county]
    
    
    print record_date, " finished!"
    

20160101  finished!
20160102  finished!
20160103  finished!
20160104  finished!
20160105  finished!
20160106  finished!
20160107  finished!
20160108  finished!
20160109  finished!
20160110  finished!
20160111  finished!
20160112  finished!
20160113  finished!
20160114  finished!
20160115  finished!
20160116  finished!
20160117  finished!
20160118  finished!
20160119  finished!
20160120  finished!
20160121  finished!
20160122  finished!
20160123  finished!
20160124  finished!
20160125  finished!
20160126  finished!
20160127  finished!
20160128  finished!
20160129  finished!
20160130  finished!
20160131  finished!
20160201  finished!
20160202  finished!
20160203  finished!
20160204  finished!
20160205  finished!
20160206  finished!
20160207  finished!
20160208  finished!
20160209  finished!
20160210  finished!
20160211  finished!
20160212  finished!
20160213  finished!
20160214  finished!
20160215  finished!
20160216  finished!
20160217  finished!
20160218  finished!
20160219  finished!


In [221]:
for hot_key,hot_value in daily_county_hottweets_dict.iteritems():    
     daily_county_hottweets_dict[hot_key]=OrderedDict(sorted(hot_value.items()))
for cold_key,cold_value in daily_county_coldtweets_dict.iteritems():    
     daily_county_coldtweets_dict[cold_key]=OrderedDict(sorted(cold_value.items()))
        
daily_county_hottweets_dict=OrderedDict(sorted(daily_county_hottweets_dict.items()))
daily_county_coldtweets_dict=OrderedDict(sorted(daily_county_coldtweets_dict.items()))

In [223]:
most_tweet_hot_county_list=list()
most_tweet_hot_county_dict=dict()
for value in daily_county_hottweets_dict.values():
    most_tweet_hot_county_list.append(max(value.iteritems(), key=operator.itemgetter(1))[0])

print "Daily summary of counties with higest amount of hot-related tweets"
count_hot_counties=Counter(most_tweet_hot_county_list)
for key, value in count_hot_counties.items():
    most_tweet_hot_county_dict[key] = value
print count_hot_counties


most_tweet_cold_county_list=list()
most_tweet_cold_county_dict=dict()
for value in daily_county_coldtweets_dict.values():
    most_tweet_cold_county_list.append(max(value.iteritems(), key=operator.itemgetter(1))[0])

print "\nDaily summary of counties with higest amount of cold-related tweets"
count_cold_counties=Counter(most_tweet_cold_county_list)
for key, value in count_cold_counties.items():
    most_tweet_cold_county_dict[key] = value
print count_cold_counties

Daily summary of counties with higest amount of hot-related tweets
Counter({'20063': 173, '31099': 66, '06037': 37, '06085': 28, '28143': 22, '35017': 20, '12086': 5, '30091': 4, '46015': 3, '30015': 2, '50015': 2, '45013': 1, '31115': 1, '08105': 1, '06081': 1})

Daily summary of counties with higest amount of cold-related tweets
Counter({'31099': 298, '28143': 10, '31115': 9, '06085': 9, '30015': 8, '24510': 6, '30091': 5, '06001': 3, '54001': 3, '24003': 2, '06081': 2, '41051': 1, '53061': 1, '25017': 1, '06071': 1, '36061': 1, '56021': 1, '41005': 1, '25027': 1, '46015': 1, '38007': 1, '23005': 1})


In [225]:
#Put 0 to the county with no tweets about weather
for key, value in daily_county_hottweets_dict.iteritems():
    county_set_with_tweets=set(value.keys())
    missing_county_list=list(set(county_list)-county_set_with_tweets)
    listofzeros = [0] * len(missing_county_list)
    value.update(zip(missing_county_list,listofzeros))
    daily_county_hottweets_dict[key]=OrderedDict(sorted(value.items()))
    
for key, value in daily_county_coldtweets_dict.iteritems():
    county_set_with_tweets=set(value.keys())
    missing_county_list=list(set(county_list)-county_set_with_tweets)
    listofzeros = [0] * len(missing_county_list)
    value.update(zip(missing_county_list,listofzeros))
    daily_county_coldtweets_dict[key]=OrderedDict(sorted(value.items()))


#### 3. Save transformed tweets data into csv file

In [522]:
#save daily value of each county into a csv file
csv_list=list()
for county_daily in daily_county_hottweets_dict.values():
    csv_list.append(county_daily.values())   

rows = zip(*csv_list)  
county_hot_df = pd.DataFrame(rows, columns=daily_county_hottweets_dict.keys())
county_hot_df['FIPS']=county_list
#with open('county_value_hot.csv','wb') as out:
#    csv_out=csv.writer(out)
#    csv_out.writerow(daily_county_hottweets_dict.keys())
#    for row in rows:
#        csv_out.writerow(row)
county_hot_df.to_csv('data\\daily dataframe\\county_values_hot.csv', index=False)
        
#save daily value of each county into a csv file
csv_list=list()
for county_daily in daily_county_coldtweets_dict.values():
    csv_list.append(county_daily.values())   

rows = zip(*csv_list)  
county_cold_df = pd.DataFrame(rows, columns=daily_county_coldtweets_dict.keys())
county_cold_df['FIPS']=county_list
#with open('county_value_cold.csv','wb') as out:
#    csv_out=csv.writer(out)
#    csv_out.writerow(daily_county_coldtweets_dict.keys())
#    for row in rows:
#        csv_out.writerow(row)
county_cold_df.to_csv('data\\daily dataframe\\county_values_cold.csv', index=False)

#### Compute potential correlation among TMAX / HOT / COLD

In [306]:
pear_hot_list=list()
spear_hot_list=list()
pear_cold_list=list()
spear_cold_list=list()

for index, row in county_hot_df.iterrows():
    nonzeros= np.count_nonzero(np.array(row.tolist()[:-1]))
    county_tmaxs = daily_county_avg_knn_dfs[daily_county_avg_knn_dfs['FIPS']==row.tolist()[-1]].values.tolist()[0][:-1]
    county_tmaxs = np.array(baldwin_tmaxs)
    if nonzeros >= round(len(county_hot_df.columns)*0.3):
        pear_hot=sp.stats.pearsonr(county_tmaxs,row.tolist()[:-1])[0]
        spear_hot=sp.stats.spearmanr(county_tmaxs,row.tolist()[:-1])[0]
    else:
        pear_hot=0
        spear_hot=0
        
    pear_hot_list.append(pear_hot)
    spear_hot_list.append(spear_hot)

for index, row in county_cold_df.iterrows():
    nonzeros= np.count_nonzero(np.array(row.tolist()[:-1]))
    county_tmaxs = daily_county_avg_knn_dfs[daily_county_avg_knn_dfs['FIPS']==row.tolist()[-1]].values.tolist()[0][:-1]
    county_tmaxs = np.array(baldwin_tmaxs)
    if nonzeros >= round(len(county_cold_df.columns)*0.3):
        pear_cold=sp.stats.pearsonr(county_tmaxs,row.tolist()[:-1])[0]
        spear_cold=sp.stats.spearmanr(county_tmaxs,row.tolist()[:-1])[0]
    else:
        pear_cold=0
        spear_cold=0
        
    pear_cold_list.append(pear_cold)
    spear_cold_list.append(spear_cold)

correlation_df = pd.DataFrame({'PEAR HOT':pear_hot_list, 'SPEAR HOT':spear_hot_list,'PEAR COLD':pear_cold_list,'SPEAR COLD':spear_cold_list})


In [307]:
print "Correlation of each county pearson tmax - hot, tmax - cold, spearman tmax- hot, tmax - cold \n "

print correlation_df.describe()

##add additional FIPS columns for map matching
county_int_list = map(int, county_list)
correlation_df['FIPS']=county_int_list

Correlation of each county pearson tmax - hot, tmax - cold, spearman tmax- hot, tmax - cold 
 
         PEAR COLD     PEAR HOT   SPEAR COLD    SPEAR HOT
count  3143.000000  3143.000000  3143.000000  3143.000000
mean     -0.103472     0.250618    -0.115460     0.165668
std       0.092353     0.133201     0.096940     0.139393
min      -0.396993    -0.247926    -0.382906    -0.265525
25%      -0.168583     0.170854    -0.183021     0.069666
50%      -0.111246     0.263223    -0.125811     0.168181
75%      -0.040727     0.342330    -0.054830     0.263693
max       0.261448     0.569367     0.296647     0.566986


In [311]:
topo_path = r'data/daily dataframe/us-10m.json'

#with open(topo_path, 'r') as f:
#    get_id = json.load(f)

#print get_id['objects']['counties']['geometries']
correlate_hot_map = folium.Map(location=[39.8282, -98.5795], zoom_start=4)
correlate_hot_map.choropleth(topo_path, data=correlation_df,columns=['FIPS','PEAR HOT'],topojson='objects.counties',
                         key_on='feature.id',
                         fill_color='YlOrRd',
                         line_weight=0.1, fill_opacity=0.3)


print "Correlation rate between TMAX and HOT counts in each county in period of 2016"
correlate_hot_map

Correlation rate between TMAX and HOT counts in each county in period of 2016


In [319]:
topo_path = r'data/daily dataframe/us-10m.json'

#with open(topo_path, 'r') as f:
#    get_id = json.load(f)

#print get_id['objects']['counties']['geometries']
correlate_map = l.Map(location=[39.8282, -98.5795], zoom_start=4)
correlate_map.choropleth(topo_path, data=correlation_df,columns=['FIPS','PEAR COLD'],topojson='objects.counties',
                         key_on='feature.id',
                         fill_color='RdYlBu',
                         line_weight=0.1, fill_opacity=0.3)


print "Correlation rate between TMAX and COLD counts in each county in period of 2016"
correlate_map

Correlation rate between TMAX and COLD counts in each county in period of 2016
